In [116]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [117]:
import seaborn as sns
import pandas as pd
import gensim
import nltk
import re
import string
from curses.ascii import isprint

In [141]:
stop_words = open('stop_words').read().split('\n')
print stop_words

['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'finds', 'first

In [119]:
stop_words.add('re')
stop_words.add('RE')
stop_words.add('Re')
stop_words.add('fw')
stop_words.add('FW')
stop_words.add('Fw')
stop_words.add('thx')
stop_words.add('pls')
# stop_words.add('call')
stop_words.add('pm')
stop_words.add('am')

In [120]:
emails_table = pd.read_csv("data/emails/Emails.csv").dropna()
person_table = pd.read_csv("data/emails/Persons.csv").dropna()
alias_table = pd.read_csv("data/emails/Aliases.csv").dropna()
email_rec_table = pd.read_csv("data/emails/EmailReceivers.csv").dropna()

In [121]:
emails_table.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
16,17,C05739573,THE YOUTH OF LIBYA,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739573...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J; Nuiand,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogirna, Ana A; Grantham, Chris W","Wednesday, September 12, 2012 9:57 PM",F-2015-04841,C05739573,05/13/2015,RELEASE IN FULL,Hope. See picture below Kamala sent.,"-ottgat,t\n4 44 4, 0\n.\n1.3.141.4443\nUNCLASS..."
17,18,C05739575,ONE MORE PHOTO,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739575...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J, Nuland,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogima, Ana A","Wednesday, September 12, 2012 10:03 PM",F-2015-04841,C05739575,05/13/2015,RELEASE IN FULL,Another photo.,c. 371.14\nUNCLASSIFIED\nU.S. Department of St...
65,66,C05739638,BRITS AND LIBYA,sullivanjj@state.gov,H,80.0,2011-04-02T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739638...,F-2015-04841,...,'sullivanjj@state.gov',H <hrod17@clintonemailcom>,Huma Abedin,"Saturday, April 02, 2011 1034 AM",F-2015-04841,C05739638,05/13/2015,RELEASE IN PART,"When is Chris Stevens going?\n---,",UNCLASSIFIED\nU.S. Department of State\nCase N...
82,83,C05739656,THE CABLE: ROMNEY TO COMPARE BENGHAZI TO ORIGI...,millscd@state.gov,H,80.0,2012-10-08T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739656...,F-2015-04841,...,"'miliscd@stategov""",H <hrod17@clintonemail.com>,"Reines, Philippe 1; Mills, Cheryl D; Adams, Da...","Monday, October 08, 2012 07:26 AM",F-2015-04841,C05739656,05/13/2015,RELEASE IN FULL,Can you call me?,UNCLASSIFIED\nU.S. Department of State\nCase N...


In [122]:
person_table.head()

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan
2,3,AP
3,4,ASUNCION
4,5,Alec


In [123]:
alias_table.head()

,Id,Alias,PersonId
0,1,111th congress,1
1,2,agna usemb kabul afghanistan,2
2,3,ap,3
3,4,asuncion,4
4,5,alec,5


In [124]:
email_rec_table.head()

,Id,EmailId,PersonId
0,1,1,80
1,2,2,80
2,3,3,228
3,4,3,80
4,5,4,80


In [125]:
print len(emails_table)
print len(person_table)
print len(alias_table)
print len(email_rec_table)

908
513
850
9306


In [126]:
emails_table['doc'] = emails_table["ExtractedSubject"] + " " + emails_table["ExtractedBodyText"]

In [127]:
def clean_text(text):
    ot = text
    # Remove any beginning or ending whitespace
    text.strip()
    # Combine conjunctions
    text = re.sub(r"'", "", text)
    # Replace punctuation with space
    text = text.translate(string.maketrans(string.punctuation,''.join(" " for char in string.punctuation)))
    # Replace all control characters
    text = ''.join(char if isprint(char) else " " for char in text)
    # Remove Digits
    text = re.sub(r"[0-9]", " ", text)
    
    
    text = text.lower()
    text_list = [token for token in text.split() if token not in stop_words and len(token) > 1]
    return text_list

In [128]:
emails_table['token_list'] = emails_table['doc'].apply(lambda s: clean_text(s))

In [129]:
dictionary = gensim.corpora.Dictionary(list(emails_table['token_list']))

[u'beneficiaries', u'stern', u'counts', u'total', u'landscape', u'sarah', u'ffyi', u'spoke', u'palestinian', u'doyle', u'keith', u'played', u'barnard', u'arms', u'call', u'recommend', u'strike', u'survive', u'preview', u'type', u'tell', u'successful', u'oscar', u'canonized', u'aware', u'spargo', u'phone', u'berlin', u'tying', u'smile', u'norways', u'hold', u'supplemental', u'stalbott', u'word', u'rights', u'mm', u'sweeten', u'declassify', u'worth', u'ms', u'experts', u'root', u'example', u'saturday', u'ramifications', u'cong', u'india', u'julissa', u'reviewing', u'scheduling', u'unsatisfying', u'david', u'worse', u'ceremony', u'pogo', u'conversation', u'provide', u'hoo', u'travel', u'nuts', u'kabul', u'allied', u'interview', u'resignation', u'ronnie', u'dunns', u'whitehead', u'description', u'typed', u'laf', u'earlier', u'confirmed', u'adoptions', u'dell', u'wrong', u'lay', u'president', u'summarize', u'law', u'types', u'effective', u'wins', u'schneider', u'thiri', u'appreciate', u'gre

In [130]:
emails_table['token_list'].head()

2                                      [chris, stevens]
16    [youth, libya, hope, picture, below, kamala, s...
17                                       [photo, photo]
65                       [brits, libya, chris, stevens]
82    [cable, romney, compare, benghazi, original, a...
Name: token_list, dtype: object

In [131]:
print dictionary.num_docs
print dictionary.num_pos
print dictionary.num_nnz

908
10313
9409


In [132]:
corpus = [dictionary.doc2bow(text) for text in emails_table['token_list']]

In [138]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [139]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [140]:
vis_data = gensimvis.prepare(lda, corpus, dictionary, R=100)
pyLDAvis.display(vis_data)

In [136]:
hdp = gensim.models.hdpmodel.HdpModel(corpus, dictionary, T=50)                                  
hdp.save('models/emails_hdp.model')

IOError: [Errno 2] No such file or directory: 'models/emails_hdp.model'

In [ ]:
alpha, beta = hdp.hdp_to_lda()
hdp_lda = gensim.models.LdaModel(id2word=hdp.id2word,num_topics=len(alpha), alpha=alpha, eta=hdp.m_eta) 
hdp_lda.expElogbeta = numpy.array(beta, dtype=numpy.float32) 

In [ ]:
vis_data = gensimvis.prepare(hdp_lda, corpus, dictionary, R=10)
pyLDAvis.display(vis_data)